In [61]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

# Prepare dataset

In [62]:
# Load all gbtc data
df_minute_gbtc_2020_12_01_2021_04_18 = pd.read_csv("../data/raw/gbtc_minute_2020-12-01_2021-04-18.csv", sep=",", index_col='date')
df_minute_gbtc_2020_12_01_2021_04_18.index.astype('datetime64[ns]')

df_gbtc_minute = df_minute_gbtc_2020_12_01_2021_04_18.drop_duplicates().sort_index(ascending=True)
df_gbtc_minute['time'] = (df_gbtc_minute['datetime'] / 1000).astype('int64')
# df_min = pd.concat([df_minute_gbtc_2020_12_01_2021_04_18, {more data in future}]).drop_duplicates().sort_index(ascending=True)

# Load all btc data
df_minute_btc_usd_minute_2020_06_01_2021_04_18 = pd.read_csv("../data/raw/btc-usd_minute_2020-06-01_2021-04-18.csv", sep=",", index_col='date')
df_minute_btc_usd_minute_2020_06_01_2021_04_18.index.astype('datetime64[ns]')
df_btc_minute = df_minute_btc_usd_minute_2020_06_01_2021_04_18.drop_duplicates()
df_btc_minute = df_btc_minute.sort_index(ascending=True)

df_btc_minute = df_btc_minute['2021-03-01': ]



In [63]:
# Join btc with gbtc on time.
df_minute = pd.merge(df_btc_minute, df_gbtc_minute, on="time", how="left")

df_minute = df_minute.rename(columns = {'low_price':'btc_low_price', 'high_price':'btc_high_price', 'close_price':'btc_close_price', 'open_price':'btc_open_price', 'volume_x':'btc_volume', 'time':'time', 'open':'gbtc_open_price', 'high':'gbtc_high_price', 
'low':'gbtc_low_price', 'close':'gbtc_close_price', 'volume_y':'gbtc_volume'})

df_minute = df_minute.drop(columns=['datetime'])
df_minute['datetime'] = df_minute['time'].apply(lambda d: datetime.utcfromtimestamp(d)) 
df_minute = df_minute.set_index('datetime')
df_minute.index.astype('datetime64[ns]')

# df_minute = df_minute.dropna()
df_minute.to_csv("../data/btc_gbtc/btc_gbtc_minute_2021.csv")